In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    torch.cuda.manual_seed_all(seed)


def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    random.seed(worker_seed)
    np.random.seed(worker_seed)

def get_optimizer(cfg,model):
    if cfg.hparam.train.optimizer == 'Adam':
        optimizer = Adam(
            params=model.parameter(),
            lr=cfg.hparam.train.lr,
        )
        return optimizer

# evaluate valid
def evaluate_one_epoch(model,loader, criterion, device):
    model.eval()
    correct = 0
    total = 0

    total_loss = 0.0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            prediceted = outputs.argmax(dim=1) # 모델이 각 이미지에 대해 선택한 클래스 번호
            correct += (prediceted == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(loader)
    accuracy = correct / total

    return avg_loss, accuracy
